In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
def make_pickles(objs):
    for filename, obj in objs:
        with open(filename, "wb") as picklefile:
            pickle.dump(obj, picklefile)

def get_pickles(pkls):
    if len(pkls) == 1:
        with open(pkls[0], "rb") as picklefile:
            objs = pickle.load(picklefile)
    else:
        objs = []
        for filename in pkls:
            with open(filename, "rb") as picklefile:
                objs.append(pickle.load(picklefile))
    return objs            

In [139]:
def dollars_to_numstring(df, columns):
    for column in columns:
        df[column] = df[column].str.replace('$','')
        df[column] = df[column].str.replace(',', '')
        df[column] = df[column].str.replace(' million','*10**6')

def to_int(item):
    if not item:
        return np.nan
    else:
        try:
            return int(item)
        except:
            try:
                return eval(item)
            except:
                return np.nan
        
def numstring_to_numeric(df, columns):
    for column in columns:
        df[column] = df[column].apply(to_int)
        
def runtime_to_minutes(runtimestring):
    if runtimestring == 'N/A':
        return np.nan
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0])*60 + int(runtime[2])
        return minutes
    except:
        raise

def to_dt(item):
    if not item or item=='TBD' or item=='N/A' or item==np.nan:
        return np.nan
    else:
        item = item.replace('Fall', 'September').replace('Winter', 'January')
    try:
        return pd.to_datetime(item)
    except:
        if len(item)==4:
            try:
                return pd.to_datetime(item, format='%Y')
            except:
                raise
        else:
            try:
                return pd.to_datetime(item,format='%B %d, %Y')
            except ValueError:
                try: 
                    return pd.to_datetime(item,format='%B %Y')
                except:
                    raise 

In [209]:
na_fills = {'page':'N/A', 'title':'N/A', 'year':np.nan, 'opening':np.nan, 'domestic':np.nan, 'budget':np.nan, 'series':'N/A',
       'distr':'N/A', 'rating':'N/A', 'release':'N/A', 'genre':'N/A', 'runtime':'N/A', 'directors':'N/A',
       'writers':'N/A', 'actors':'N/A', 'producers':'N/A', 'opening_adj':np.nan, 'domestic_adj':np.nan,
       'budget_adj':np.nan}

df = get_pickles(['df.pkl'])
df.fillna(value=na_fills, inplace=True)

cols = ['opening','domestic','opening_adj', 'domestic_adj','budget', 'budget_adj']
dollars_to_numstring(df, cols)
numstring_to_numeric(df, cols)
df['year'] = pd.to_datetime(df['year']).dt.year
df['release'] = df['release'].apply(to_dt)
df['rating'] = df['rating'].apply(lambda x:x.replace('GP','PG'))
df['runtime'] = df['runtime'].apply(runtime_to_minutes)
df.dropna(subset=['release'],inplace=True)
df.drop(df[df.release > pd.to_datetime('10/1/2017')].index, inplace=True)
df.drop(df[df.release < pd.to_datetime('10/1/1977')].index, inplace=True)

In [51]:
df_numbers = get_pickles(['df_numbers.pkl'])

cols = ['Production Budget', 'Domestic Gross', 'Worldwide Gross']
dollars_to_numstring(df_numbers, cols)
numstring_to_numeric(df_numbers, cols)
df_numbers['Release Date'] = pd.to_datetime(df_numbers['Release Date'])
df_numbers['Year'] = df_numbers['Release Date'].dt.year
df_numbers.dropna(subset=['Release Date'], inplace=True)
df_numbers.drop(df_numbers[df_numbers['Release Date'] > pd.to_datetime('10/1/2017')].index, inplace=True)
df_numbers.drop(df_numbers[df_numbers['Release Date'] < pd.to_datetime('10/1/1977')].index, inplace=True)

In [52]:
df_AZ = get_pickles(['df_AZ.pkl'])

cols = ['Total Gross', 'Total Theaters', 'Opening', 'Opening Theaters']
dollars_to_numstring(df_AZ, cols)
numstring_to_numeric(df_AZ, cols)
df_AZ['Open'] = df_AZ['Open'].apply(to_dt)

In [238]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15771 entries, 0 to 17552
Data columns (total 19 columns):
page            15771 non-null object
title           15771 non-null object
year            15771 non-null float64
opening         13821 non-null float64
domestic        15122 non-null float64
budget          3112 non-null float64
series          15771 non-null object
distr           15771 non-null object
rating          15771 non-null object
release         15771 non-null datetime64[ns]
genre           15771 non-null object
runtime         15322 non-null float64
directors       15771 non-null object
writers         15771 non-null object
actors          15771 non-null object
producers       15771 non-null object
opening_adj     13275 non-null float64
domestic_adj    15141 non-null float64
budget_adj      3112 non-null float64
dtypes: datetime64[ns](1), float64(8), object(10)
memory usage: 3.0+ MB


---

`!pip install fuzzywuzzy`

In [7]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [8]:
matches_by_year = {}
for year in range(1977, 2018):
    for query in df_numbers[(df_numbers.Year==year)].Movie:
        matches_by_year[(query,year)] = process.extractOne(query, df[(df.year==year)].title, score_cutoff=70)
    print(year, end=' ')

1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 

In [9]:
make_pickles([('matches_by_year.pkl', matches_by_year)])

In [10]:
def get_key(Movie, Year):
    try:
        return matches_by_year[(Movie,Year)][0]
    except TypeError:
        return 'N/A'

In [53]:
df_numbers['title'] = [get_key(movie, year) for (movie,year) in np.array(df_numbers[['Movie', 'Year']])]

In [54]:
df_numbers.drop(df_numbers[df_numbers.duplicated(subset=['Movie', 'Year'], keep='last')].index,inplace=True)

---

In [217]:
df_merge = df.merge(df_AZ, how='left', left_on='page', right_on='Link').merge(df_numbers,how='left', \
                                                                              left_on=['title', 'year'], \
                                                                              right_on=['title', 'Year'])

In [218]:
df_merge.columns

Index(['page', 'title', 'year', 'opening', 'domestic', 'budget', 'series',
       'distr', 'rating', 'release', 'genre', 'runtime', 'directors',
       'writers', 'actors', 'producers', 'opening_adj', 'domestic_adj',
       'budget_adj', 'Link', 'Title', 'Studio', 'Total Gross',
       'Total Theaters', 'Opening', 'Opening Theaters', 'Open', 'Release Date',
       'Movie', 'Production Budget', 'Domestic Gross', 'Worldwide Gross',
       'Year'],
      dtype='object')

In [219]:
df_merge.drop(['page','opening','domestic','budget','directors','writers','actors','producers','domestic_adj','budget_adj',\
              'Link','Title','Studio','Total Gross','Total Theaters','Opening','Open','Release Date','Movie','Domestic Gross',\
              'Worldwide Gross','Year'],axis=1, inplace=True)

In [220]:
df_merge.count()

title                15976
year                 15976
series               15976
distr                15976
rating               15976
release              15976
genre                15976
runtime              15522
opening_adj          13429
Opening Theaters     14321
Production Budget     4947
dtype: int64

In [221]:
df_merge.runtime.median()

100.0

In [222]:
df_merge.runtime.fillna(100,inplace=True)

In [223]:
df_merge.dropna(subset=['opening_adj','Production Budget', 'Opening Theaters',],inplace=True)

---

In [224]:
df_merge.columns

Index(['title', 'year', 'series', 'distr', 'rating', 'release', 'genre',
       'runtime', 'opening_adj', 'Opening Theaters', 'Production Budget'],
      dtype='object')

In [225]:
distr_df = df_merge.groupby('distr')[['opening_adj','title']].agg({'opening_adj':'mean','title':'count'}).round()\
                                            .sort_values(by='opening_adj', ascending=False).head(20).reset_index()
distr_df['distr_20'] = 1
distr_df.drop(['opening_adj','title'], axis=1, inplace=True)
df_merge = df_merge.merge(distr_df, how='left', on='distr')
df_merge.distr_20.fillna(0, inplace=True)

In [226]:
def genre_grp(item):
    genres = ['Horror', 'Sci-Fi', 'Thriller', 'Action', 'Comedy', 'Drama']
    for genre in genres:
        if genre in item:
            return genre
    else:
        return 'Other'    

df_merge['genre_grp'] = df_merge['genre'].apply(genre_grp)

In [227]:
rating_df = df_merge.groupby('rating').opening_adj.mean().round().sort_values(ascending=False).reset_index().head()
rating_df['rating_grp'] = rating_df['rating']
rating_df.drop(['opening_adj'],axis=1, inplace=True)
df_merge = df_merge.merge(rating_df, how='left', on='rating')

In [228]:
df_merge['Franchise'] = [int(series!='N/A') for series in df_merge.series]

In [229]:
df_merge['Season'] = df_merge['release'].map(lambda x:['Winter','Summer','Spring','Fall'][int(x.month//3)-1])

In [230]:
df_merge.columns

Index(['title', 'year', 'series', 'distr', 'rating', 'release', 'genre',
       'runtime', 'opening_adj', 'Opening Theaters', 'Production Budget',
       'distr_20', 'genre_grp', 'rating_grp', 'Franchise', 'Season'],
      dtype='object')

In [232]:
df_merge.drop(['series','distr','rating','release','genre'],axis=1, inplace=True)

In [234]:
make_pickles([('df_merge.pkl',df_merge)])

---

In [233]:
df_merge.columns

Index(['title', 'year', 'runtime', 'opening_adj', 'Opening Theaters',
       'Production Budget', 'distr_20', 'genre_grp', 'rating_grp', 'Franchise',
       'Season'],
      dtype='object')